# Analyse des performances des étudiants


## À propos

### 1. Description:

L'ensemble de données sur les performances des étudiants est un ensemble de données conçu pour examiner les facteurs qui influencent les performances scolaires des étudiants. L'ensemble de données se compose de 10_000 dossiers d'étudiants, chaque dossier contenant des informations sur divers prédicteurs et un indice de performance.

### 2. Variables:

- Heures étudiées: le nombre total d'heures passées à étudier par chaque étudiant.
- Scores précédents: les scores obtenus par les étudiants lors des tests précédents.
- Activités parascolaires: si l'étudiant participe à des activités parascolaires (oui ou non).
- Heures de sommeil: le nombre moyen d'heures de sommeil de l'étudiant par jour.
- Exemples de questionnaires pratiqués: le nombre de questionnaires types que l'étudiant a pratiqués.

## Étape 1 : Exploration des données

Nous chargeons l'ensemble de données (format : csv), optimisons la structure des données pour mieux s'adapter à la mémoire et nous explorons ses métadonnées, sa forme et ses statistiques..

In [3]:
import pandas as pd
import plotly.express as px
DATASET_PATH = "./Student_Performance.csv"

In [4]:
df = pd.read_csv(DATASET_PATH).drop("Extracurricular Activities", axis=1)
f"L'ensemble de données se compose de {df.shape[0]} lignes et {df.shape[1]} colonnes"

"L'ensemble de données se compose de 10000 lignes et 5 colonnes"

In [5]:
df[df.select_dtypes('int64').columns] = df.select_dtypes('int64').astype('int8')
df[df.select_dtypes('float64').columns] = df.select_dtypes('float64').astype('float32')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Hours Studied                     10000 non-null  int8   
 1   Previous Scores                   10000 non-null  int8   
 2   Sleep Hours                       10000 non-null  int8   
 3   Sample Question Papers Practiced  10000 non-null  int8   
 4   Performance Index                 10000 non-null  float32
dtypes: float32(1), int8(4)
memory usage: 78.2 KB


### Visualisation de données

In [6]:
agg_df = df.groupby("Hours Studied")['Performance Index'].mean()

fig = px.bar(agg_df, y="Performance Index", x=agg_df.index, title="Average Performance Distribution by Hours Studied", range_y=[40, 70])
fig.show()

In [7]:
agg_df = df.groupby("Sleep Hours")['Performance Index'].mean()

fig = px.bar(agg_df, y="Performance Index", x=agg_df.index, title="Average Performance Distribution by Sleep Hours", range_y=[53, 57])
fig.show()

In [8]:
agg_df = df.groupby("Sample Question Papers Practiced")['Performance Index'].mean()

fig = px.bar(agg_df, y="Performance Index", x=agg_df.index, title="Average Performance Distribution by Sample Question Papers Practiced", range_y=[52, 58])
fig.show()

### Etude de la colonne cible

À partir du tableau de corrélation, nous concluons que le meilleur prédicteur de l'indice de performance est les scores antérieurs de l'étudiant `Previous Scores` car il a la valeur de corrélation la plus élevée.

In [9]:
df.corr()

,Hours Studied,Previous Scores,Sleep Hours,Sample Question Papers Practiced,Performance Index
Hours Studied,1.000000,-0.012390,0.001245,0.017463,0.373730
Previous Scores,-0.012390,1.000000,0.005944,0.007888,0.915189
Sleep Hours,0.001245,0.005944,1.000000,0.003990,0.048106
Sample Question Papers Practiced,0.017463,0.007888,0.003990,1.000000,0.043268
Performance Index,0.373730,0.915189,0.048106,0.043268,1.000000


## Étape 2 : Modélisation

Le KPI `Performance Index` étudié est un nombre décimal, le modèle final doit donc être un modèle de régression.
Pour trouver le modèle le mieux adapté, nous devons créer un pipeline qui divise l'ensemble de données en KFolds aléatoires, appliquer des transformateurs puis valider les résultats sur plusieurs types de modèles et avec une hyperparamétrisation.

In [10]:
%pip install scikit-learn

In [20]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import StackingRegressor

### Modèles

In [21]:
linear = Pipeline(steps=[
    ("poly", PolynomialFeatures()),
    ("linear_regression", LinearRegression())
])
sgd = SGDRegressor()
tree = DecisionTreeRegressor()
estimators=[
  ("linear_regression", linear),
  ("sgd", sgd),
  ("tree", tree)
]

### Hyperparamétres

In [17]:
linear_params = {
    'poly__degree': [2,3,4,5]
}
sgd_params = {
    'loss': ['squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
    'penalty': ['l1', 'l2', 'elasticnet'],
    'alpha': [0.1, 0.01, 0.0001, 0.00001]
}
tree_params = {
    'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
    'max_depth': [2, 5, 7]
}
param_grid = [
    linear_params,
    sgd_params,
    tree_params
]
param_grid = [[params] for params in param_grid]

### Pipeline

In [14]:
encoding_steps = [
    ("onehotencoder", OneHotEncoder(), ["Extracurricular Activities"])
]

### Cross Validation

In [15]:
def run_cv(estimator_name, estimator, params):
  pipeline = Pipeline([
      #("transformer", ColumnTransformer(transformers=encoding_steps)),
      ("estimator", estimator)
  ])
  cv = GridSearchCV(estimator, params,
                    cv=KFold(6),
                    scoring=[
                        "r2",
                        "neg_mean_absolute_error",
                        "neg_mean_squared_error",
                        "neg_root_mean_squared_error"],
                    refit="r2")

  print(f"Lancer entrainement pour {estimator_name}...")
  cv.fit(
      X=df.drop(["Performance Index"], axis=1),
      y=df["Performance Index"]
  )
  result_df = pd.DataFrame(cv.cv_results_)
  result_df = result_df.sort_values(by="mean_score_time")
  #print(result_df.columns)
  print("Resultats")
  print(result_df[["params", "mean_test_r2", "std_test_r2"]].head(5))
  return cv.best_score_, cv.best_params_, cv.best_estimator_

### Entraînement et résultats

In [22]:
scores = []
for estimator_, params in zip(estimators, param_grid):
  estimator_name, estimator = estimator_
  score = run_cv(estimator_name, estimator, params)
  scores.append((estimator_name, score))

Lancer entrainement pour linear_regression...
Resultats
                params  mean_test_r2  std_test_r2
3  {'poly__degree': 5}      0.988294     0.000190
2  {'poly__degree': 4}      0.988379     0.000191
1  {'poly__degree': 3}      0.988431     0.000195
0  {'poly__degree': 2}      0.988451     0.000231
Lancer entrainement pour sgd...
Resultats
                                               params  mean_test_r2  \
29  {'alpha': 0.0001, 'loss': 'huber', 'penalty': ...      0.914544   
41  {'alpha': 1e-05, 'loss': 'huber', 'penalty': '...      0.913368   
27  {'alpha': 0.0001, 'loss': 'huber', 'penalty': ...      0.913421   
15  {'alpha': 0.01, 'loss': 'huber', 'penalty': 'l1'}      0.914903   
16  {'alpha': 0.01, 'loss': 'huber', 'penalty': 'l2'}      0.912060   

    std_test_r2  
29     0.002804  
41     0.007987  
27     0.003399  
15     0.001442  
16     0.002443  
Lancer entrainement pour tree...
Resultats
                                            params  mean_test_r2  std_test

In [38]:
max_score = 0
params = {}
estimator = ""
for estimator_name, score in scores:
   value, param_grid, _ = score

   max_score = max(max_score, value)
   if max_score == value:
      params = param_grid
      estimator = estimator_name

print(estimator, f"{max_score:.1%}", params)

linear_regression 98.8% {'poly__degree': 2}
